# Web Scrape - Ceticismo Político

Objetivo: extrair a headline, descrição e link das principais notícias do portal Ceticismo Político afim de identificar padrões referentes à Fake News.

Assumimos que este site publica notícias consideradas fake news de acordo com esta publicação: https://www.issoenoticia.com.br/artigo/projeto-da-usp-lista-10-maiores-sites-de-falsas-noticias-no-brasil

Fonte: https://ceticismopolitico.com/

## Importe das bibliotecas necessárias

In [34]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

## Definição das funções de parse da URL

Função que retorna o HTML da url fornecida

In [35]:
def parse_url(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html5lib')
    
    return html

Função que extrai título, link, data e resumo do post. Como cada site possui uma estrutura diferente, não é possível generalizar essa função.

In [49]:
def get_posts(url, pages=1):
    post_data = []

    html = parse_url(url)
    posts = html.find_all('div', 'item-details')

    for i, post in enumerate(posts):
        title = post.find('h1', 'entry-title')
        date = post.find('div', 'td-module-meta-info')
        excerpt = post.find('div', 'td-post-text-content')

        if (title is not None) & (date is not None) & (excerpt is not None):
            excerpt_p = ''
            for paragraph in excerpt.find_all('p', recursive=False):
                excerpt_p = excerpt_p + paragraph.get_text()

            post_data.append([title.find('a').get_text(), title.find('a').get('href'), date.find('time', 'entry-date').get('datetime'), excerpt_p])
    if pages > 1:
        for p in tqdm(range(2, pages)):
            html = parse_url(url + 'page/' + str(p) + '/')
            posts = html.find_all('div', 'item-details')

            for i, post in enumerate(posts):
                title = post.find('h1', 'entry-title')
                date = post.find('div', 'td-module-meta-info')
                excerpt = post.find('div', 'td-post-text-content')

                if (title is not None) & (date is not None) & (excerpt is not None):
                    excerpt_p = ''
                    for paragraph in excerpt.find_all('p', recursive=False):
                        excerpt_p = excerpt_p + paragraph.get_text()
                    post_data.append([title.find('a').get_text(), title.find('a').get('href'), date.find('time', 'entry-date').get('datetime'), excerpt_p])

    return np.array(post_data)

## Captura os posts do site

In [54]:
posts = get_posts('https://ceticismopolitico.com/', pages=1)

## Criação e tratamento do dataframe

In [51]:
dataframe = pd.DataFrame(posts, columns=['title', 'url', 'date', 'excerpt'])

Verifica a qualidade dos dados gerados

In [52]:
dataframe.head()

,title,url,date,excerpt
0,Professora que colocou camisinha com a boca em...,https://ceticismopolitico.com/2017/12/06/profe...,2017-12-06T20:09:53+00:00,Um vídeo compartilhado nas redes sociais que m...
1,Kátia Abreu diz que vai colocar sua expulsão e...,https://ceticismopolitico.com/2017/11/30/katia...,2017-11-30T21:16:15+00:00,A senadora Kátia Abreu (sem partido-TO) disse ...
2,Blog esquerdista dá a entender que reclamar de...,https://ceticismopolitico.com/2017/11/28/blog-...,2017-11-28T19:07:54+00:00,"Um texto de Jorge Gauthier, para o blog Me Sal..."
3,"Alckmin diz que por ele PSDB “desembarca”, mas...",https://ceticismopolitico.com/2017/11/28/alckm...,2017-11-28T18:40:04+00:00,"O governador de São Paulo, Geraldo Alckmin, as..."
4,Cara de pau não tem limites: Zé Celso aciona M...,https://ceticismopolitico.com/2017/11/28/cara-...,2017-11-28T18:01:46+00:00,O Teatro Oficina procurou o Ministério Público...


In [53]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
title      90 non-null object
url        90 non-null object
date       90 non-null object
excerpt    90 non-null object
dtypes: object(4)
memory usage: 2.9+ KB


## Saída para CSV

In [194]:
dataframe.to_csv('ceticismopolitico.csv', encoding='utf-8')